<a href="https://colab.research.google.com/github/seokhee516/Project2-Credit-Scoring-System/blob/main/%EA%B2%B0%EA%B3%BC%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 문제정의

이번 프로젝트 주제는 **"씬파일러를 위한 신용평가 예측모형"** 입니다.  
 씬파일러(Thin filer)란, 금융 거래가 적거나 없어 관련 서류가 얇은 금융고객을 의미합니다. 주로 사회초년생, 주부, 은퇴자들이 이에 속해 있으며, 이들은 중저신용등급으로 책정되어 제도권 금융을 이용할 기회가 줄어들게 됩니다.
 이처럼 금융이력이 부족한 이들의 채무불이행을 예측하고, 더 나아가 신파일러들의 금융 불이익을 해소 할 수 있는 신용평가를 제공하는 것이 이번 프로젝트의 목적입니다.




프로젝트에 사용할 데이터는 **"Lending Club(렌딩 클럽)"** 데이터입니다.  
렌딩 클럽은 미국 유명 P2P 대출 업체로서, 신용등급이 낮아 제도권 금융기관에서 대출을 받을 수 없는 씬파일러에게 대안금융 역할을 하고 있습니다. 캐글에서 제공하고 있는 'Lending Club 2007-2020Q3' 데이터세트의 2018년부터 2020년 3분기 데이터를 활용하여 본 프로젝트를 진행하겠습니다.  



렌딩 클럽 데이터는 대출상태(loan_status)를 상환, 유지, 연체, 회수불능 등 8개로 분류하였습니다. 이를 **정상(상환, 유지)과 불량(연체, 회수불능)으로 이진 분류 문제**를 풀어내겠습니다.

# 2. EDA 및 데이터 전처리

## 2.1 데이터 및 모듈 불러오기

In [134]:
! pip install --upgrade imbalanced-learn

In [135]:
!pip uninstall scikit-learn -y
!pip install -U scikit-learn

Found existing installation: scikit-learn 1.0.1
Uninstalling scikit-learn-1.0.1:
  Successfully uninstalled scikit-learn-1.0.1
  Using cached scikit_learn-1.0.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (23.2 MB)


In [136]:
!pip install category_encoders

In [137]:
!pip install catboost

In [138]:
!pip install lightgbm

In [139]:
! pip install pdpbox

In [140]:
! pip install shap

In [141]:
import pandas as pd
import numpy as np
import missingno as msno

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.model_selection import RandomizedSearchCV

from pdpbox.pdp import pdp_isolate, pdp_plot
from pdpbox.pdp import pdp_interact, pdp_interact_plot
import shap


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')

In [142]:
df = pd.read_csv('LendingClub_Loan_status_2018-2020Q3.csv').iloc[:,1:]
print(df.shape)
df.head()

FileNotFoundError: ignored

In [ ]:
msno.bar(df=df.iloc[:, :], color=(0.8, 0.5, 0.2))

## 2.2 데이터 전처리

1. 대출 후 생성 변수 제거
2. 최빈값의 빈도가 99% 이상인 범주형 변수 제거
3. 범주형 데이터 타입 정리
4. 범주의 수가 50개 이상인 변수 제거
5. 결측값 절반 이상 칼럼 제거

### 2.2.1 대출 후 생성 변수 제거  
렌딩클럽 데이터에는 대출 생성 전에는 존재하지 않는 대출 수 생성 변수가 존재한다. 대출 후 생성 변수는 모형에 학습될 경우, 과적합될 가능성이 높아진다.   
recoveries와 같은 변수는 추심된 금액 중 회복된 금액을 설명한다. 이와 유사하게 대출 후 생성 변수 30개를 제거한다. 

In [ ]:
recoveries_list = ['out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d',
'last_credit_pull_d', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'hardship_flag',
'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount',
'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd',
'hardship_loan_status', 'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount',
'hardship_last_payment_amount']

In [ ]:
df.drop(columns=recoveries_list,axis=1, inplace=True)

### 2.2.2 최빈값의 빈도가 99% 이상인 범주형 변수 제거  
최빈값의 빈도가 99% 이상인 범주형 변수는 모형에 학습된다고 해도 결과 변수에 영향을 주지못한다.


In [ ]:
mode_99_list = ['pymnt_plan', 'policy_code', 'debt_settlement_flag', 
                'num_tl_120dpd_2m', 'acc_now_delinq', 'num_tl_30dpd']

In [ ]:
df.drop(columns=mode_99_list,axis=1, inplace=True)

### 2.2.3 범주형 데이터 타입 정리
emp_length는 재직 기간을 의미하는 변수로, 범주의 수가 10개 이상으로 dummy encoding을 진행할 경우 차원이 10개 이상 늘어나게 된다. 따라서 이를 방지하기 위해 범주군을 묶어주었다.
- less than 3 years: 1년 이하 ~ 3년
- more than 4 years and less than 9 years: 4년 ~ 9년
- 10+ years: 10년 이상
- unemployed: nan

In [ ]:
df.loc[(df['emp_length']== '< 1 year')|(df['emp_length']== '1 year')|(df['emp_length']== '2 years')|(df['emp_length']== '3 years'),'emp_length'] = 'less than 3 years'
df.loc[(df['emp_length']== '4 years')|(df['emp_length']== '5 years')|(df['emp_length']== '6 years')|(df['emp_length']== '7 years')|(df['emp_length']== '8 years')|(df['emp_length']== '9 years'),'emp_length'] = 'more than 4 years and less than 9 years'
df['emp_length'].fillna('unemployed', inplace=True)

purpose의 경우 대출 목적의 의미를 가진 변수로, 이 또한 범주군을 묶어주었다.
- General loan debt: car, home_improvement, house, major_purchase, medical, moving,
other, renewable_energy, small_business, vacation, wedding 
- credit_card: credit_card
- debt_consolidation: debt_consolidation

In [ ]:
df.loc[(df['purpose']== 'car')|(df['purpose']== 'home_improvement')|(df['purpose']== 'house')|(df['purpose']== 'major_purchase')|
       (df['purpose']== 'medical')|(df['purpose']== 'moving')|(df['purpose']== 'other')|(df['purpose']== 'renewable_energy')|(df['purpose']== 'small_business')|
       (df['purpose']== 'vacation')|(df['purpose']== 'wedding'),'purpose'] = 'General loan debt'

int_rate, revol_util 변수는 %가 붙어 있으므로, 수치형으로 변환해주었다.

In [ ]:
df[['int_rate', 'revol_util']] = df[['int_rate', 'revol_util']].replace('%','', regex=True).apply(pd.to_numeric)

### 2.2.4 범주의 수가 50개 이상인 변수 제거

In [ ]:
selected_cols = df.select_dtypes(include='object')
colnames = selected_cols.columns.tolist()
labels = selected_cols.nunique()

selected_features = labels[labels >= 50].index.tolist()

In [ ]:
df.drop(columns=selected_features,axis=1, inplace=True)

### 2.2.5 결측값 절반 이상 칼럼 제거

In [ ]:
df.dropna(thresh=int(len(df) * 0.5), axis=1, inplace=True)

### 2.2.6 변수 생성
loan_status 변수를 정상(0)과 불량(1)으로 이진 분류해주었다.
- 0: Fully Paid, Current
- 1: Late (31-120 days), Charged Off, In Grace Period, Late (16-30 days), Default, Issued

In [ ]:
df.loc[(df['loan_status']== 'Fully Paid')|(df['loan_status']== 'Current'),"loan_payment"]= 0

In [ ]:
df.loc[(df['loan_status']== 'Late (31-120 days)')|(df['loan_status']== 'Charged Off')|
       (df['loan_status']== 'In Grace Period')|(df['loan_status']== 'Late (16-30 days)')|
       (df['loan_status']== 'Default')|(df['loan_status']== 'Issued')|
       (df['loan_status']== 'Late (31-120 days)'),'loan_payment'] = 1

## 2.3 EDA

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
msno.bar(df=df.iloc[:, :], color=(0.8, 0.5, 0.2))

In [ ]:
sns.kdeplot(x=df["annual_inc"])

In [ ]:
sns.boxplot(df['annual_inc'])

In [ ]:
sns.scatterplot(data = df, x ='annual_inc' , y ='loan_amnt')

In [ ]:
sns.barplot(x='home_ownership', y = 'loan_amnt', data = df)

In [ ]:
sns.barplot(x='loan_status', y = 'loan_amnt', data = df)

In [ ]:
sns.boxplot(y = df['loan_amnt'], x = df['loan_status'])

In [ ]:
sns.countplot(x = df['loan_payment'])

In [ ]:
sns.barplot(x='loan_payment', y = 'loan_amnt', data = df)

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df.drop(columns=['loan_status','id','issue_d','grade']).corr())

# 3. 모델링

In [ ]:
# 평가지표 함수
def model_evaluation(label, predict):
    cf_matrix = confusion_matrix(label, predict)
    Accuracy = (cf_matrix[0][0] + cf_matrix[1][1]) / sum(sum(cf_matrix))
    Precision = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[0][1])
    Recall = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[1][0])
    F1_Score = (2 * Recall * Precision) / (Recall + Precision)
    print("Model_Evaluation with Label: 1")
    print("Accuracy: {:.1%}".format(Accuracy))
    print("Precision: {:.1%}".format(Precision))
    print("Recall: {:.1%}".format(Recall))
    print("F1-Score: {:.1%}".format(F1_Score))

## 3.1 타겟 지정 및 데이터 세트 분할

In [ ]:
target= 'loan_payment'

# test set 만들기
test = df[(df['issue_d'] == 'Sep-2020') | (df['issue_d'] == 'May-2020')]
train = df.drop(test.index)

# validation set 만들기
train, val = train_test_split(train, train_size = 0.8, stratify=train[target], random_state=10)

In [ ]:
# target 지정 및 데이터 세트 분할
# loan_payment: Target, id: Always unique, issue_d: Date, grade: Duplicative of sub_grade
features = train.drop(columns=['loan_status','loan_payment','id','issue_d','grade']).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

In [ ]:
print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)
print('X_val shape', X_val.shape)
print('y_val shape', y_val.shape)
print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

In [ ]:
# Data Processing Pipeline
processor = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(strategy='mean')
)
X_train_processed = processor.fit_transform(X_train)
X_val_processed = processor.transform(X_val)

# Scailing
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_processed)
X_val_scaled = scaler.transform(X_val_processed)

## 3.2 기본값 모델링

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
# Majority class baseline
# 기준모델 학습세트
major = y_train.mode()[0]
y_train_pred = [major] *len(y_train)
print("기준모델 학습세트 평가")
model_evaluation(y_train, y_train_pred)
print("\n")

# 기준모델 검증세트
y_val = val[target]
y_val_pred = [major] * len(y_val)
print("기준모델 검증세트 평가")
model_evaluation(y_val, y_val_pred)

In [ ]:
# Logistic Regression
lr = LogisticRegression(random_state=10, n_jobs=-1)
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_val_scaled)

print("로지스틱 검증세트 평가")
model_evaluation(y_val, y_pred_lr)

In [ ]:
# Decision Tree Classifier
dtc = DecisionTreeClassifier(random_state=1, max_depth=6,
                           min_samples_leaf=3, min_samples_split=2)
dtc.fit(X_train_processed, y_train)
y_pred_dtc = dtc.predict(X_val_processed)

print("결정트리 검증세트 평가")
model_evaluation(y_val, y_pred_dtc)

In [ ]:
# Random Forest Classifier
rfc = RandomForestClassifier(random_state=1, n_jobs=-1)
rfc.fit(X_train_processed, y_train)
y_pred_rfc = rfc.predict(X_val_processed)

print("렌덤포레스트 검증세트 평가")
model_evaluation(y_val, y_pred_rfc)

In [ ]:
# XGB Classifier
xgb = XGBClassifier(random_state=10, n_jobs=-1)
xgb.fit(X_train_processed, y_train)
y_pred_xgb = xgb.predict(X_val_processed)

print("XGBoost 검증세트 평가")
model_evaluation(y_val, y_pred_xgb)

In [ ]:
# LGBM Classifier
lgb = LGBMClassifier(random_state=10, n_jobs=-1)
lgb.fit(X_train_processed, y_train);
y_pred_lgb = lgb.predict(X_val_processed)

print("LGBM 검증세트 평가")
model_evaluation(y_val, y_pred_lgb)

In [ ]:
# CatBoostClassifier
cbc = CatBoostClassifier(random_state=10)
cbc.fit(X_train_processed, y_train);
y_pred_cbc = cbc.predict(X_val_processed)

print("CatBoost 검증세트 평가")
model_evaluation(y_val, y_pred_cbc)

## 3.4 SMOTE를 이용한 Oversampling 후 모델링

In [ ]:
print("OverSampling 전, '불량(1)' 라벨의 수: {}".format(sum(y_train == 1)))
print("OverSampling 전, '정상(0)' 라벨의 수: {}".format(sum(y_train == 0)))

In [ ]:
# SMOTE 알고리즘 활용하여 Oversampling 진행
sm = SMOTE(random_state = 10)
X_train_res, y_train_res = sm.fit_resample(X_train_processed, y_train.ravel()) 

In [ ]:
print("OverSampling 후, '불량(1)' 라벨의 수: {}".format(sum(y_train_res == 1)))
print("OverSampling 후, '정상(0)' 라벨의 수: {}".format(sum(y_train_res == 0)))

In [ ]:
pd.Series(y_train_res).value_counts(normalize=True)

In [ ]:
sns.countplot(x=pd.Series(y_train_res))

In [ ]:
# Logistic Regression Over Sampling
lr2 = LogisticRegression(random_state=10, n_jobs=-1)
lr2.fit(X_train_res, y_train_res)
y_pred_lr2 = lr2.predict(X_val_scaled)

print("로지스틱 검증세트 평가")
model_evaluation(y_val, y_pred_lr2)

In [ ]:
# Decision Tree Classifier Over Sampling
dtc2 = DecisionTreeClassifier(random_state=10, max_depth=6,
                           min_samples_leaf=3, min_samples_split=2)
dtc2.fit(X_train_res, y_train_res)
y_pred_dtc2 = dtc2.predict(X_val_processed)

print("결정트리 검증세트 평가")
model_evaluation(y_val, y_pred_dtc2)

In [ ]:
# Random Forest Classifier Over Sampling
rfc2 = RandomForestClassifier(max_depth=12, min_samples_leaf=2, min_samples_split= 18, n_estimators=300,
                           criterion='entropy', max_features='auto',n_jobs=-1, random_state=10)
rfc2.fit(X_train_res, y_train_res)
y_pred_rfc2 = rfc2.predict(X_val_processed)

print("렌덤포레스트 검증세트 평가")
model_evaluation(y_val, y_pred_rfc2)

In [ ]:
# XGBClassifier Over Sampling
xgb2 = XGBClassifier(random_state=10, n_jobs=-1)
xgb2.fit(X_train_res, y_train_res)
y_pred_xgb2 = xgb2.predict(X_val_processed)

print("XGBoost 검증세트 평가")
model_evaluation(y_val, y_pred_xgb2)

In [ ]:
# LGBM Classifier Over Sampling
lgb2 = LGBMClassifier(random_state=10, n_jobs=-1)
lgb2.fit(X_train_res, y_train_res);
y_pred_lgb2 = lgb2.predict(X_val_processed)

print("LGBM 검증세트 평가")
model_evaluation(y_val, y_pred_lgb2)

In [ ]:
# CatBoostClassifier Over Sampling
cbc2 = CatBoostClassifier(random_state=10)
cbc2.fit(X_train_res, y_train_res);
y_pred_cbc2 = cbc2.predict(X_val_processed)

print("CatBoost 검증세트 평가")
model_evaluation(y_val, y_pred_cbc2)

# 3.5 Hyperparameter Tuning 후 모델링

In [ ]:
vc = y_train.value_counts().to_list()
ratio = float(vc[0]/vc[1])
ratio

In [ ]:
# XGBClassifier Hyperparameter Tuning
xgb3 = XGBClassifier(random_state=10, n_jobs=-1,n_estimators=500, max_depth=7, 
                     scale_pos_weight=ratio) # weight 조절

eval_set = [(X_train_processed, y_train), 
            (X_val_processed, y_val)]

xgb3.fit(X_train_processed, y_train,
         eval_set=eval_set, eval_metric='error', early_stopping_rounds=50);
y_pred_xgb3 = xgb3.predict(X_val_processed)


print("XGBoost 검증세트 평가")
model_evaluation(y_val, y_pred_xgb3)

# 3.6 Over Sampling + Hyperparameter Tuning 후 모델링

In [ ]:
# RandomForestClassifier Over Sampling + Hyperparameter Tuning
rfc3 = RandomForestClassifier(random_state=10, n_jobs=-1, n_estimators=1000, max_depth=7)
rfc3.fit(X_train_res, y_train_res)
y_pred_rfc3 = rfc3.predict(X_val_processed)

print("랜덤포레스트 검증세트 평가")
model_evaluation(y_val, y_pred_rfc3)

In [ ]:
# XGBClassifier  Over Sampling + Hyperparameter Tuning
xgb4 = XGBClassifier(random_state=10, n_jobs=-1,n_estimators=500, max_depth=7, 
                     scale_pos_weight=ratio) # weight 조절

eval_set = [(X_train_res, y_train_res), 
            (X_val_processed, y_val)]

xgb4.fit(X_train_res, y_train_res,
         eval_set=eval_set, eval_metric='error', early_stopping_rounds=50);
y_pred_xgb4 = xgb4.predict(X_val_processed)


print("XGBoost 검증세트 평가")
model_evaluation(y_val, y_pred_xgb4)

# 4. 결과해석

In [ ]:
feature_names = X_train.columns.tolist()
pd.Series(xgb3.feature_importances_, feature_names).sort_values(ascending=False)

In [ ]:
lr_pipe = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    LogisticRegression(random_state=10, n_jobs=-1)
)
lr_pipe.fit(X_train, y_train);

In [ ]:
model_lr = lr_pipe.named_steps['logisticregression']
enc = lr_pipe.named_steps['ordinalencoder']
encoded_columns = enc.transform(X_val).columns
coefficients = pd.Series(model_lr.coef_[0], encoded_columns)
plt.figure(figsize=(15,15))
coefficients.sort_values().plot.barh()
plt.show()

In [ ]:
coefficients = pd.Series(np.around(model_lr.coef_,5)[0], encoded_columns)
coefficients.sort_values(ascending=False)